In [1]:
import os
import glob
import pandas as pd

# -----------------------------
# CONFIGURACIÓN
# -----------------------------
RUTA_PADRE = "../data"
RUTA_ECU911_RAW = os.path.join(RUTA_PADRE, "raw", "ecu911", "dataset")
PATRON_CSV = os.path.join(RUTA_ECU911_RAW, "*.csv")

SALIDA_LIMPIA = os.path.join(RUTA_PADRE, "interim", "ecu911", "ecu911_limpio.csv")
os.makedirs(os.path.dirname(SALIDA_LIMPIA), exist_ok=True)

archivos_csv = glob.glob(PATRON_CSV)
print(f"Archivos CSV encontrados: {len(archivos_csv)}")
print("Ejemplo:", archivos_csv[:3])

Archivos CSV encontrados: 10
Ejemplo: ['../data\\raw\\ecu911\\dataset\\incidentes_abril_2025.csv', '../data\\raw\\ecu911\\dataset\\incidentes_agosto_2025.csv', '../data\\raw\\ecu911\\dataset\\incidentes_enero_2025.csv']


In [2]:
def normalizar_columnas (df: pd.DataFrame):
  df = df.copy()
  df.columns = (
    df.columns.astype(str)
    .str.replace("ï»¿", "", regex=False)
    .str.strip()
    .str.lower()
  )
  return df

def limpiar_texto_basico(serie: pd.Series) -> pd.Series:
  s = serie.astype(str).str.strip()
  s = s.replace({"": pd.NA, "nan": pd.NA, "none": pd.NA, "null": pd.NA})
  return s

def estandarizar_cod_parroquia(serie: pd.Series) -> pd.Series:
  s = limpiar_texto_basico(serie)
  s = s.str.replace(".0", "", regex=False)
  s = s.str.zfill(6)
  return s

def parsear_fecha(serie: pd.Series) -> pd.Series:
    # Convierte a datetime de forma robusta
    return pd.to_datetime(serie, errors="coerce", dayfirst=True)


def limpiar_lat_lon(df: pd.DataFrame, col_lat="lat", col_lon="lon") -> pd.DataFrame:
    """
    Limpia lat/lon si existen: convierte coma->punto, numérico, elimina nulos,
    elimina ceros y rangos inválidos.
    """
    df = df.copy()

    if col_lat not in df.columns or col_lon not in df.columns:
        print(f"Aviso: no existen columnas {col_lat}/{col_lon}. Se omite limpieza lat/lon.")
        return df

    for col in [col_lat, col_lon]:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace(",", ".", regex=False)
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

    antes = len(df)
    df = df.dropna(subset=[col_lat, col_lon])

    # Quitar ceros (como en tu ejemplo)
    df = df[(df[col_lat] != 0) & (df[col_lon] != 0)]

    # Rangos válidos geográficos
    df = df[(df[col_lat].between(-90, 90)) & (df[col_lon].between(-180, 180))]

    print(f"Limpieza lat/lon: eliminados {antes - len(df)} registros inválidos.")
    return df


def leer_csv_ecu911(path_csv: str) -> pd.DataFrame:
    """
    Lee CSV ECU911 con separador ';', normaliza columnas y aplica filtro
    (si aplica) sin crear features.
    """
    df = pd.read_csv(
        path_csv,
        sep=";",
        encoding="UTF-8",
        on_bad_lines="skip",
        low_memory=False
    )
    df = normalizar_columnas(df)
    return df



In [3]:
dfs = []
errores = []

print("Cargando archivos ECU911... esto puede tardar unos segundos...")

for archivo in archivos_csv:
    try:
        df_tmp = leer_csv_ecu911(archivo)
        # df_tmp["__source_file"] = os.path.basename(archivo)  # trazabilidad (opcional)
        dfs.append(df_tmp)
    except Exception as e:
        errores.append((archivo, str(e)))

if errores:
    print(f"Archivos con error: {len(errores)}")
    print("Ejemplo:", errores[0])

df_911 = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

print(f"Registros totales unificados: {len(df_911)}")
print(f"Columnas: {list(df_911.columns)}")
df_911.head()

Cargando archivos ECU911... esto puede tardar unos segundos...
Registros totales unificados: 2679924
Columnas: ['fecha', 'provincia', 'canton', 'cod_parroquia', 'parroquia', 'servicio', 'subtipo']


,fecha,provincia,canton,cod_parroquia,parroquia,servicio,subtipo
0,1/4/2025,SANTO DOMINGO DE LOS TSACHILAS,SANTO DOMINGO,230150.0,"SANTO DOMINGO DE LOS COLORADOS, CABECERA CANTO...",Seguridad Ciudadana,Violencia contra la mujer o miembros del núcle...
1,1/4/2025,SANTO DOMINGO DE LOS TSACHILAS,SANTO DOMINGO,230150.0,"SANTO DOMINGO DE LOS COLORADOS, CABECERA CANTO...",Seguridad Ciudadana,Patrullaje policial en el sector solicitado
2,1/4/2025,SANTO DOMINGO DE LOS TSACHILAS,SANTO DOMINGO,230150.0,"SANTO DOMINGO DE LOS COLORADOS, CABECERA CANTO...",Servicios Municipales,Libadores - GAD
3,1/4/2025,SANTO DOMINGO DE LOS TSACHILAS,SANTO DOMINGO,230150.0,"SANTO DOMINGO DE LOS COLORADOS, CABECERA CANTO...",Seguridad Ciudadana,Patrullaje policial en el sector solicitado
4,1/4/2025,SANTO DOMINGO DE LOS TSACHILAS,SANTO DOMINGO,230150.0,"SANTO DOMINGO DE LOS COLORADOS, CABECERA CANTO...",Seguridad Ciudadana,Robo de motos


In [4]:
# -----------------------------
# 1) FILTRO OPCIONAL (si forma parte de limpieza por alcance)
# -----------------------------
if "servicio" in df_911.columns:
    antes = len(df_911)
    df_911["servicio"] = limpiar_texto_basico(df_911["servicio"])
    df_911 = df_911[df_911["servicio"] == "Seguridad Ciudadana"]
    print(f"Filtro servicio=Seguridad Ciudadana: eliminados {antes - len(df_911)}")

# -----------------------------
# 2) LIMPIEZA DE FECHA
# -----------------------------
if "fecha" in df_911.columns:
    # Parsear fecha original
    df_911["fecha_dt"] = parsear_fecha(df_911["fecha"])
    
    antes = len(df_911)
    df_911 = df_911.dropna(subset=["fecha_dt"])
    print(f"Fechas inválidas eliminadas: {antes - len(df_911)}")
    
    # Eliminar columna original y renombrar
    df_911 = df_911.drop(columns=["fecha"])
    df_911 = df_911.rename(columns={"fecha_dt": "fecha"})
    
    # Asegurar tipo datetime
    df_911["fecha"] = pd.to_datetime(df_911["fecha"])
else:
    print("Aviso: no existe columna 'fecha'.")


# -----------------------------
# 3) LIMPIEZA DE COD_PARROQUIA (solo formato, sin cruce)
# -----------------------------
# Nota: tras normalizar columnas, normalmente quedará 'cod_parroquia' aunque viniera como 'Cod_Parroquia'
if "cod_parroquia" in df_911.columns:
    df_911["cod_parroquia"] = estandarizar_cod_parroquia(df_911["cod_parroquia"])
    antes = len(df_911)
    df_911 = df_911.dropna(subset=["cod_parroquia"])
    print(f"Códigos de parroquia vacíos eliminados: {antes - len(df_911)}")
else:
    # Esto es útil para diagnosticar cómo se llama realmente
    posibles = [c for c in df_911.columns if "parroq" in c or "parro" in c]
    print("Aviso: no existe 'cod_parroquia'. Candidatas:", posibles)



Filtro servicio=Seguridad Ciudadana: eliminados 905747
Fechas inválidas eliminadas: 0
Códigos de parroquia vacíos eliminados: 204


In [5]:
# Intenta limpiar lat/lon con nombres típicos: lat/lon
df_911 = limpiar_lat_lon(df_911, col_lat="lat", col_lon="lon")

# Si tu dataset usa "latitud"/"longitud" en vez de lat/lon, usa esto:
# df_911 = limpiar_lat_lon(df_911, col_lat="latitud", col_lon="longitud")

print(f"Registros después de limpieza lat/lon (si aplicó): {len(df_911)}")


Aviso: no existen columnas lat/lon. Se omite limpieza lat/lon.
Registros después de limpieza lat/lon (si aplicó): 1773973


In [6]:
try:
    df_911.to_csv(SALIDA_LIMPIA, index=False, encoding="utf-8")
    print("Dataset limpio guardado correctamente.")
    print("Salida:", SALIDA_LIMPIA)
    print("Registros finales:", len(df_911))
except Exception as e:
    print("Error guardando el CSV:", e)


Dataset limpio guardado correctamente.
Salida: ../data\interim\ecu911\ecu911_limpio.csv
Registros finales: 1773973
